<a href="https://colab.research.google.com/github/kellysolow/OFA-HFeng-NICE/blob/master/OFA_prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
csv_path = './drive/MyDrive/NICE/'
val_set = pd.read_csv(f'{csv_path}/nice-val-5k.csv')
val_set = pd.read_csv(f'{csv_path}/nice-test.csv')

Mounted at /content/drive


이미지 데이터, tsv 형식(img_id, '\t', img_content)(base64 인코딩):

In [ ]:
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import base64

result = []
for pic_id in  tqdm(val_set['public_id'], total = len(val_set)):
  img = Image.open(f'{csv_path}cvpr-nice-test/{pic_id}.jpg') # path to file
  img_buffer = BytesIO()
  img.save(img_buffer, format=img.format)
  byte_data = img_buffer.getvalue()
  base64_str = base64.b64encode(byte_data) # bytes
  base64_str = base64_str.decode("utf-8") # str

  resultStr = str(pic_id) + '\t' + base64_str
  result.append(resultStr)
  

100%|██████████| 21377/21377 [1:32:48<00:00,  3.84it/s]


In [ ]:
import csv
resultDF = pd.DataFrame(result)


resultDF.to_csv(f'{csv_path}/train_image(need to delete backslash).tsv',sep='\t', encoding = 'utf-8', index = False, header=None, quoting=csv.QUOTE_NONE,escapechar='\\' ) 
#str에서 큰따옴표가 앞뒤로 저장되는 개같은현상
#vs에서 저장후 재업로드..

캡션 데이터, jsonl 형식:
{"image_id": "007c720f1d8c04104096aeece425b2d5", "text": ["性感名媛蕾丝裙，尽显优雅撩人气质", "衣千亿

In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import jsonlines

# 예시 데이터
caption_result = []
imsi_result = {}
for index, i in enumerate(val_set['public_id']):
  imsi_result['image_id'] = str(i)
  imsi_result['text'] = [val_set['caption_gt'][index]]
  a = imsi_result.copy()
  caption_result.append(a)



# 파일에 JSONNL 형식으로 데이터 저장
with jsonlines.open(f'{csv_path}/train_caption.jsonl', mode='w') as writer:
    for item in caption_result:
        writer.write(item)
